In [1]:
import os

In [2]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [3]:
os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

### Data Ingestion Configuration
Organize and store configuration settings for the data ingestion process.

#### Configuration entities

In [ ]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

#### Configuration manager

In [5]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

#### Data Ingestion Component

In [11]:
from urllib import request
import zipfile
from cnn_classifier import logger
from cnn_classifier.utils.utilities import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """Initialize DataIngestion with the given configuration."""
        self.config = config
    
    def download_data(self):
        """
        Download the data file from the source URL if it does not already exist locally.

        Returns:
            str: Path to the downloaded (or existing) local data file.

        Raises:
            Exception: If an error occurs during download.
        """
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"Downloaded {filename} with following info:\n{headers}")
            else:
                logger.info(f"Data already exists at {self.config.local_data_file} ({get_size(Path(self.config.local_data_file))}), skipping download")
            
            return self.config.local_data_file
        except Exception as e:
            logger.error(f"Error downloading data: {e}")
            raise
    
    def extract_zip_file(self):
        """Extract the contents of the downloaded zip file into the configured directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file_ref:
            zip_file_ref.extractall(unzip_path)
        
        # Rename top-level folder: train/ or Train/ -> images/
        base = Path(unzip_path)
        src = None
        for candidate in [base / "train", base / "Train"]:
            if candidate.exists() and candidate.is_dir():
                src = candidate
                break
        if src is None:
            return
        
        dst = base / "images"
        
        # If already renamed in a previous run, do nothing
        if dst.exists() and dst.is_dir():
            return
        
        # Avoid accidental overwrite
        if dst.exists():
            raise FileExistsError(f"Cannot rename to '{dst}': path exists.")
        
        src.rename(dst)

#### Data Ingesiton Pipeline

In [21]:
# Create the data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_cofig = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_cofig)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-20 15:42:36,274]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-20 15:42:36,275]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-20 15:42:36,276]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts
[2025-12-20 15:42:36,276]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/data_ingestion
[2025-12-20 15:42:44,854]: cnnClassiferLogger - INFO - 3410142567 - Downloaded artifacts/data_ingestion/data.zip with following info:
Connection: close
Content-Length: 278791023
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'
Content-Type: application/zip
ETag: "6641a21b18310e23c1b848c99c0552b66eaf58bb3821f52563aecec448e0704e"
Last-Modified: Sat, 20 Dec 2025 21:42:37 GMT
Strict-Transport-Security: max-age=31557600
timing-allo